In [4]:
from dotenv import load_dotenv
from langchain import PromptTemplate
from langchain.chains import SequentialChain, LLMChain
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
import os
import json
import pandas as pd

In [5]:
load_dotenv()

True

In [6]:
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY2")

In [7]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.8,
)

C:\Users\rajun\AppData\Local\Temp\ipykernel_23016\1124066220.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [8]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple question",
        "options":{
            "a":"choice answer",
            "b":"choice answer",
            "c":"choice answer",
            "d":"choice answer",
        },
        "correct_answer":"correct answer"
    }
}

In [9]:
TEMPLATE=""" 
text={text}
subject={subject}
tone={tone}
number={number}

Instructios:
You are to generate a quiz in a {tone} tone based on the subject "{subject}".
the quiz should contain {number} multiple-choice question.
Each question should have:
- a clear and concise question text
- four option lebeled A, B, C, D
- a correct answer key

output must be in the following JSON format:
{response_json}

"""

In [10]:
quiz_prompt = PromptTemplate(
    input_variables=["text", "subject", "number", "tone", "response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_prompt,
    output_key="quiz",
    verbose=True
)

C:\Users\rajun\AppData\Local\Temp\ipykernel_23016\3773648120.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(


In [12]:
TEMPLATE = """
You are an expert English grammarian and writer. Given a multiple-choice quiz for {subject} students,
you need to evaluate the complexity of the questions and provide a complete analysis of the quiz.

- Use **no more than 50 words** for the complexity analysis.
- If any question does not align with the cognitive and analytical abilities of the students,
  revise only those questions and adjust the tone to better suit the appropriate difficulty level.

Quiz MCQs:
{quiz}

Please provide your evaluation and the updated quiz (if needed) from the perspective of an expert English writer:
"""


In [13]:
quiz_evalution_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE,
)

In [14]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evalution_prompt,
    output_key="review",
    verbose=True
)

In [15]:
gen_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "subject", "number", "tone", "response_json"],
    output_variables=["quiz", 'review'],
    verbose=True
)

In [16]:
from pathlib import Path
file_path = r'E:\learning\Gen-AI\mcqgen\data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
NUMBER=5
SUBJECT="Data science"
TONE="Hard"

In [19]:
with get_openai_callback() as cb:
    response = gen_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\rajun\AppData\Local\Temp\ipykernel_23016\2310826965.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = gen_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
text=Data science is an interdisciplinary academic field[1] that uses statistics, scientific computing, scientific methods, processes, algorithms and systems to extract or extrapolate knowledge and insights from noisy, structured, and unstructured data.[2]

Data science also integrates domain knowledge from the underlying application domain (e.g., natural sciences, information technology, and medicine).[3] Data science is multifaceted and can be described as a science, a research paradigm, a research method, a discipline, a workflow, and a profession.[4]

Data science is a "concept to unify statistics, data analysis, informatics, and their related methods" to "understand and analyze actual phenomena" with data.[5] It uses techniques and theories drawn from many fields within the context of mathematics, statistics, computer science, information science, and domain knowledge.[6] Howeve

In [27]:
print(f"Tokens used: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total cose: ${cb.total_cost}")

Tokens used: 2101
Prompt Tokens: 1151
Completion Tokens: 950
Total cose: $0.00074265


In [28]:
quiz_str = response["quiz"].strip("`").replace("json", "").strip()

In [29]:
quiz_dict= json.loads(quiz_str)
quiz_dict

{'1': {'mcq': 'What is the primary purpose of data science?',
  'options': {'a': 'To create programming languages',
   'b': 'To extract knowledge and insights from data',
   'c': 'To develop hardware solutions',
   'd': 'To manage databases'},
  'correct_answer': 'b'},
 '2': {'mcq': 'Which of the following fields does NOT directly contribute to data science?',
  'options': {'a': 'Statistics',
   'b': 'Philosophy',
   'c': 'Computer Science',
   'd': 'Information Science'},
  'correct_answer': 'b'},
 '3': {'mcq': 'According to Jim Gray, how is data science characterized in relation to traditional paradigms of science?',
  'options': {'a': 'As a purely theoretical field',
   'b': 'As the first paradigm of science',
   'c': 'As a fourth paradigm that is data-driven',
   'd': 'As a subset of empirical science'},
  'correct_answer': 'c'},
 '4': {'mcq': 'Which of the following skills is essential for a data scientist?',
  'options': {'a': 'Creative writing',
   'b': 'Statistical knowledge',


In [30]:
quiz_table = []
for key, value in quiz_dict.items():
    mcq = value['mcq']
    options = " || ".join(
        [
            f"{option} : {option_value} "
            for option, option_value in value["options"].items()
        ]
    )
    correct_answer = value["correct_answer"]
    quiz_table.append({"MCQ":mcq, "Choices":options, "Correct_Answer":correct_answer})

In [31]:
df = pd.DataFrame(quiz_table)
df

,MCQ,Choices,Correct_Answer
0,What is the primary purpose of data science?,a : To create programming languages || b : To...,b
1,Which of the following fields does NOT directl...,a : Statistics || b : Philosophy || c : Comp...,b
2,"According to Jim Gray, how is data science cha...",a : As a purely theoretical field || b : As t...,c
3,Which of the following skills is essential for...,a : Creative writing || b : Statistical knowl...,b
4,What role does human-computer interaction play...,a : It is irrelevant to data analysis || b : ...,b


In [32]:
df.to_csv("data_science.csv", index=False)